In [ ]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter
from email.mime import image
from imutils import face_utils
import argparse
import imutils
import time
from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install mediapipe
import mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
matlist = [Path(f).stem for f in glob.glob("/content/drive/MyDrive/AFLW2000/*.mat")]
imglist = [Path(f).stem for f in glob.glob("/content/drive/MyDrive/AFLW2000/*.jpg")]
imglist[0]


'3231797341_1'

In [ ]:
droped_imgs = []
c=0

def get_landmarks(img,path):
    X=[]
    Y=[]
   # print(os.path.join('AFLW2000/', image_name).type)
    # global c ,X,Y
    faceModule = mediapipe.solutions.face_mesh
# loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the imag
        image = cv2.imread(path+img+'.jpg')
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None:
          # looping over the faces in the image
          if len(results.multi_face_landmarks)==1 :
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    shape = image.shape
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])
                    X.append(relative_x)
                    Y.append(relative_y)      
        return X,Y

In [ ]:
a,b = get_landmarks('3231797341_1','/content/drive/MyDrive/AFLW2000/')
print (b)

[308, 285, 294, 267, 278, 270, 253, 268, 240, 231, 199, 311, 314, 315, 332, 335, 338, 342, 349, 289, 292, 252, 268, 270, 271, 271, 265, 252, 251, 255, 259, 276, 362, 267, 281, 274, 293, 309, 316, 315, 320, 319, 322, 338, 286, 280, 255, 274, 291, 288, 299, 273, 243, 248, 237, 241, 252, 331, 346, 293, 294, 330, 329, 245, 294, 240, 235, 211, 240, 222, 254, 253, 313, 317, 321, 294, 329, 333, 329, 290, 322, 319, 317, 350, 344, 340, 336, 333, 333, 334, 336, 339, 315, 315, 292, 332, 332, 297, 298, 295, 280, 287, 292, 223, 230, 238, 344, 233, 217, 204, 272, 282, 262, 264, 269, 287, 290, 285, 284, 280, 275, 270, 257, 303, 265, 290, 280, 284, 265, 293, 268, 283, 330, 260, 277, 355, 365, 310, 348, 267, 370, 292, 286, 278, 268, 267, 335, 317, 377, 373, 370, 214, 374, 265, 263, 261, 265, 258, 258, 258, 261, 263, 267, 268, 299, 309, 292, 302, 247, 361, 366, 372, 358, 259, 267, 369, 376, 325, 334, 336, 339, 342, 348, 325, 325, 325, 322, 316, 262, 253, 255, 324, 335, 250, 355, 265, 262, 259, 278, 362,

In [ ]:
# read_mat function to return marks pitch, yaw, roll
def fileRead(img,path):
  mat_file = sio.loadmat(path+img+'.mat')
  # extracting the labels 3 angels
  pose_para = mat_file["Pose_Para"][0][:3]
  pitch = pose_para[0]
  yaw = pose_para[1]
  roll = pose_para[2]
  return [yaw, pitch, roll, img]

In [ ]:
# for i in matlist :
  


In [ ]:
marks_x = ['x' + str(i) for i in range(468 )]
marks_y = ['y' + str(i) for i in range(468 )]
features = marks_x + marks_y
labels = ['pitch', 'yaw', 'roll']
df = pd.DataFrame(columns=features + labels)

def bulid_dataFrame(path):
    global df
    # path = 'AFLW2000/image00002.jpg'
    X_point,Y_point, pitch, yaw, roll = fileRead(path)
    print('x_axis: ', len(X_point))
    print('x_axis: ', len(Y_point))
   # print('labels: ', len(labels))
    df_x = pd.DataFrame(X_point,columns=marks_x)
    df_x = df_x.astype(float)
    df_y = pd.DataFrame(Y_point,columns=marks_y)
    df_y = df_y.astype(float)
    # landmarks_x consists of strings from 0 to 667

    # flaten the landmarks into multiple values and add to the dataframe
  
    

    #print (df)
    #df = df.append(df2)

    # df = df.


In [ ]:
def get_list_from_filenames(file_path):
    # input:    relative path to .mat file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines


In [ ]:

# x_points_col = ['X{}'.format(i) for i in range(468)]
# y_points_col = ['Y{}'.format(i) for i in range(468)]

#def bulid_dataFrame(path):
imagePath = [Path(f).stem for f in glob.glob('/content/drive/MyDrive/AFLW2000/'+'*.jpg')]
X_points = []
Y_points = []
labels = []
for img in imagePath:
  # print(img)
  labels.append(fileRead(img,'/content/drive/MyDrive/AFLW2000/'))
  X, Y = get_landmarks(img,'/content/drive/MyDrive/AFLW2000/')
  X_points.append(X)
  Y_points.append(Y)
df1= pd.DataFrame(X_points)
df2= pd.DataFrame(Y_points)
df3 = pd.DataFrame(labels,columns=['yaw', 'pitch', 'roll', 'image'])
df = pd.concat([df1,df2,df3],axis=1)

In [ ]:
 df = df.dropna()

In [ ]:
df.to_csv('data.csv',index = False)

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3833 entries, 0 to 3836
Columns: 940 entries, 0 to image
dtypes: float64(939), object(1)
memory usage: 27.5+ MB


In [ ]:
result = pd.read_csv("data.csv")


In [ ]:
marks_x = ['x' + str(i) for i in range(468)]
marks_y = ['y' + str(i) for i in range(468)]
features = marks_x + marks_y
result.columns.values[0:936]=features


In [ ]:
# import numpy as np

# for i in range(468):
#     xstr = 'x'+str(i)
#     ystr = 'y'+str(i)
#     pointstr = 'point'+str(i)
#     result[pointstr] = np.sqrt((result[xstr] -result['x99'])**2 +(result[ystr]-result['y99'])**2)
#     if i != 99:
#         result.drop([xstr,ystr], axis=1, inplace=True)
# result.drop(['x99', 'y99'], axis=1, inplace=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [ ]:
result.iloc[:,-4:-1]

,yaw,pitch,roll
0,-0.142117,0.032600,-0.226494
1,-0.092551,-0.152758,0.119983
2,0.041173,-0.452514,-0.036319
3,0.040558,0.281446,0.022014
4,0.310935,0.116551,0.287020
...,...,...,...
3828,-0.246066,-0.111348,0.001073
3829,0.050202,-0.187037,-0.064051
3830,0.042420,-0.231104,-0.141656
3831,0.304315,-0.054411,-0.131326


In [ ]:
X = result.drop(['yaw', 'pitch', 'roll','image'], axis=1)
y_yaw = result['yaw']
y_pitch = result['pitch']
y_roll = result['roll']
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-4] ,df.iloc[:,-4:-1], test_size=0.20, random_state=42)
# X_train_yaw, X_test_m yaw, y_train_yaw, y_test_yaw = train_test_split(
#     df.iloc[:,:-4] ,df.iloc[:,-4:-1], test_size=0.20, random_state=42)
# X_train_pitch, X_test_pitch, y_train_pitch, y_test_pitch = train_test_split(
#     X, y_pitch, test_size=0.20, random_state=42)
# X_train_roll, X_test_roll, y_train_roll, y_test_roll = train_test_split(
#     X, y_roll, test_size=0.20, random_state=42)


In [ ]:
y_test_yaw = y_test['yaw']
y_test_pitch = y_test['pitch']
y_test_roll = y_test['roll']
y_train_yaw = y_train['yaw']
y_train_pitch = y_train['pitch']
y_train_roll = y_train['roll']


In [ ]:
# model_yaw = LinearRegression().fit(X_train, y_train_yaw)
# model_pitch = LinearRegression().fit(X_train, y_train_pitch)
# model_roll = LinearRegression().fit(X_train, y_train_roll)
# y_pred_yaw = model_yaw.predict(X_test)
# y_pred_pitch = model_pitch.predict(X_test)
# y_pred_roll = model_roll.predict(X_test)


In [ ]:
# print(r2_score(y_test_yaw, y_pred_yaw))
# print(r2_score(y_test_pitch, y_pred_pitch))
# # print(r2_score(y_test_roll, y_pred_roll))

0.9551237067990284
0.7370325716516732
0.9238002566974959


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
model_yaw = RandomForestRegressor(n_estimators=50).fit(X_train, y_train_yaw)
model_pitch = RandomForestRegressor(n_estimators=50).fit(X_train, y_train_pitch)
model_roll = LinearRegression().fit(X_train, y_train_roll)
y_pred_yaw = model_yaw.predict(X_test)
y_pred_pitch = model_pitch.predict(X_test)
y_pred_roll = model_roll.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score


In [ ]:
print('Pitch:')
print('Mean Squared Error:', mean_squared_error(y_test_pitch, y_pred_pitch))
print('R2 Score:', r2_score(y_test_pitch, y_pred_pitch))
print('Mean Absolute Error:', mean_absolute_error(y_test_pitch, y_pred_pitch))
print('Explained Variance Score:',
      explained_variance_score(y_test_pitch, y_pred_pitch))

print("\n Yaw:")
print('Mean Squared Error:', mean_squared_error(y_test_yaw, y_pred_yaw))
print('R2 Score:', r2_score(y_test_yaw, y_pred_yaw))
print('Mean Absolute Error:', mean_absolute_error(y_test_yaw, y_pred_yaw))
print('Explained Variance Score:',
      explained_variance_score(y_test_yaw, y_pred_yaw))

print("\n Roll:")
print('Mean Squared Error:', mean_squared_error(y_test_roll, y_pred_roll))
print('R2 Score:', r2_score(y_test_roll, y_pred_roll))
print('Mean Absolute Error:', mean_absolute_error(y_test_roll, y_pred_roll))
print('Explained Variance Score:',
      explained_variance_score(y_test_roll, y_pred_roll))


Pitch:
Mean Squared Error: 0.005466233513943145
R2 Score: 0.8073054734157707
Mean Absolute Error: 0.054921574760837026
Explained Variance Score: 0.8073522513743097

 Yaw:
Mean Squared Error: 0.0019059848879505337
R2 Score: 0.9772018035598402
Mean Absolute Error: 0.026709227381046675
Explained Variance Score: 0.9772255995921306

 Roll:
Mean Squared Error: 0.0016592148085268254
R2 Score: 0.9238002566975001
Mean Absolute Error: 0.02999742846032293
Explained Variance Score: 0.9243124741742316


In [ ]:
#drow yaw and pitch and roll on the image opencv



In [ ]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)



In [ ]:
def get_landmark(img):
    x_axis = []
    y_axis = []
    faceModule = mediapipe.solutions.face_mesh
    
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if results.multi_face_landmarks != None:
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y

                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])

                    x_axis.append(relative_x)
                    y_axis.append(relative_y)
                    # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                    #cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
                #if draw == True:
                  #cv2_imshow(image)
    return x_axis, y_axis


In [ ]:
vid = cv2.VideoCapture('/content/drive/MyDrive/hossam2.mp4')
width= int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

if (vid.isOpened()== False): 
    print("Error opening video file")

frames = list()

In [ ]:
faceModule = mediapipe.solutions.face_mesh   
while vid.isOpened():                                                          
    val, frame = vid.read()                                                     

    if val == True:                                                             
      with faceModule.FaceMesh(static_image_mode=True) as face:
     
        results = face.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.multi_face_landmarks != None:
          for face in results.multi_face_landmarks:
              
            X = list()                                                          
            Y = list()                                                         

            for landmark in face.landmark:                                     
                X.append(landmark.x )                      
                Y.append(landmark.y) 
                        
            X = np.array(X)                                                   
            Y = np.array(Y)
            featuresnp = np.hstack([X,Y]).reshape(1,-1)
            features=pd.DataFrame(featuresnp)
            marks_x = ['x' + str(i) for i in range(468)]
            marks_y = ['y' + str(i) for i in range(468)]
            featur = marks_x + marks_y
            features.columns.values[0:]=featur
            pitch = model_pitch.predict(features)
            yaw=model_yaw.predict(features)
            roll=model_roll.predict(features)
            draw_axis(frame,pitch,yaw,roll,X[2],Y[2])
            
            frames.append(frame)                                                
    else:
      break                                                                       
vid.release()  
# Closes all the frames
cv2.destroyAllWindows()
print("Number of Detected Frames = ",len(frames))

Number of Detected Frames =  265


In [ ]:

out = cv2.VideoWriter('out.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))
for i in range(len(frames)):
    out.write(frames[i])
out.release()

In [ ]:
from google.colab import files
files.download('out.mp4')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>